<div style="text-align:center;">
    <img src="http://www.cs.wm.edu/~rml/images/wm_horizontal_single_line_full_color.png">
    <h1>CSCI 416-01: Introduction to Machine Learning</h1>
    <h1>Fall 2025</h1>
    <h1>Decision trees</h1>
</div>    

# Contents

* [Decision trees](#Decision-trees)
* [Features of decision trees](#Features-of-decision-trees)
* [Constructing a decision tree](#Constructing-a-decision-tree)
* [Measures of homogeneity](#Measures-of-homogeneity)
    * [Misclassification rate](#Misclassification-rate)
    * [Gini index](#Gini-index)
    * [Entropy](#Entropy)
    * [The two-class case](#The-two-class-case)
* [Information gain](#Information-gain)
* [Fisher's iris dataset](#Fisher's-iris-dataset)
* [Building a decision tree in Scikit-Learn](#Building-a-decision-tree-in-Scikit-Learn)
* [Continuous features](#Continuous-features)
* [Missing values](#Missing-values)
* [Overfitting](#Overfitting)
    * [The bias-variance tradeoff](#The-bias-variance-tradeoff)
* [Pruning](#pruning)
* [Regression trees](#Regression-trees)

In [ ]:
%matplotlib inline
from IPython.display import Image

# Fun with scikit-learn <a id="scikit-learn"></a>

For most of this course we will use the Python [Scikit-Learn ML library](https://scikit-learn.org/stable/).  It can be found in [PyPl under the name <code>scikit-learn</code>](https://pypi.org/project/scikit-learn/).  It is also part of the Anaconda distribution.

This notebook also requires the <code>matplotlib</code> and <code>graphviz</code> modules.  The <code>graphviz</code> module, in turn, needs you to install the [<code>DOT</code> package](https://github.com/xflr6/graphviz?tab=readme-ov-file#installation).

# Decision trees

Decision trees are easy to understand and help reveal the structure of our data.

<img src="http://www.cs.wm.edu/~rml/teaching/csci416/images/classification_tree.png" style="width:500px;"/>

Some well-known examples are 
* Classification and regression trees (CART) (Breiman, Friedman, Olshen, and Stone)
* ID3 $\Longrightarrow$ C4.5 $\Longrightarrow$ C5.0 (Quinlan)

For now, we will look at decision trees for classification.

# Features of decision trees

Advantages of decision trees:
- They are easy to explain to others.
- They can be displayed graphically, which makes them easier to understand.
- They easily handle qualitiative variables (e.g., weak and strong for wind).
- They arguably reflect human decision making.

Disadvantages of decision trees:
- They arguably reflect human decision making.
- They generally do not have the same level of predictve accuracy as other methods.

# Constructing a decision tree

Variants of **Hunt's method** are commonly used to construct decision trees classifiers.

Let $T$ be the set of training cases from which we will build the tree, and let $C_{1}, \ldots, C_{K}$ be the classes.  There are three possibilities.
1. $T$ contains one or more cases, all of which belong to a single class $C_{j}$.  The decision tree for $T$ is a leaf identifying $C_{j}$.
2. $T$ contains no cases.  The decision tree is a leaf, but the associated class must be determined from information other than $T$ (e.g., the most frequent class at the leaf's parent).
3. $T$ contains cases that belong to a mixture of classes.  In this case, the idea is to refine $T$ into subsets of cases that tend towards single-class collections of cases.
    - Using a single feature we choose a test with mutually exclusive outcomes $O_{1}, \ldots, O_{N}$, $n > 1$.
    - Partition $T$ into subsets $T_{1}, \ldots, T_{N}$, where $T_{i}$ contains all the cases in $T$ with outcome $O_{i}$.
    - This yields a decision node, with a branch for each outcome.
    - Now apply this procedure recursively to $T_{1}, \ldots, T_{N}$.

Most decision tree constructions use a greedy approach to choosing tests.

Tests are selected to partition the current training set on the basis on a local measure of progress.
  
The goal of a test is to produce more homogeneous groups of cases $T_{1}, \ldots, T_{N}$.

To do so, we need a measure of homogeneity.

# Measures of homogeneity

Given a set of cases $S$, let $p_{i} = p_{i}(S)$ be the proportion of cases in $S$ that belong to class $i$.

Let $m = m(S)$ be the class that makes up the majority of cases in $S$.

We will classify the cases in $S$ to be instances of class $m$.

Three commonly used measures of homogeneity are
1. **Misclassification rate**: $$1 - p_{m}$$
2. **Gini index**: $$\sum_{\mbox{all classes $k$}}\!\!\!\! p_{k} (1 - p_{k})$$
3. **Entropy**: $$\sum_{\mbox{all classes $k$}}\!\!\!\! -p_{k} \log_{2} p_{k}$$

The entropy is sometimes scaled by 1/2 so that it has the same maximum as the other two.

For all three, values closer to zero mean greater homogeneity; larger values mean greater heterogeneity.

## Misclassification rate

If we stopped subdividing $S$ and left it a leaf, then the logical thing to do is to assign the majority class associated with the leaf.

The **misclassification rate** is the proportion of cases in $S$ incorrectly assigned to the majority class $m$.  It tells us the proportion of training cases in the leaf that are misclassified.

Example: suppose there are only two outcomes for testing on an feature (as with "Windy").

Suppose $S$ is initially a collection of 14 examples, 9 cats and 5 dogs.

Then $p_{m} = 9/14$, and the misclassification rate is 
$$
  1 - \frac{9}{14} = \frac{5}{14} = 0.36.
$$

## Gini index

If instead of classifying all cases in $S$ to be the majority class, for each class $k$ we could assign them to class $k$ with probabililty $p_{k}$.

The probability that you are assigned to class $k$ by mistake is then
$$
  p_{k} \sum_{j \neq k} p_{j} = p_{k} (1-p_{k}).
$$
Summing over all classes give the Gini index as a measure of the training error rate.

In our cats/dogs example, the Gini index is
$$ 
  \frac{9}{14} (1 - \frac{9}{14}) + \frac{5}{14} (1 - \frac{5}{14}) = \frac{90}{196} = 0.46.
$$

## Entropy

Entropy is used in the decision tree package C5.0 and its predecessors, C4.5 and ID3.

In our cats/dogs example, we will scale the entropy by 1/2 so that is more readily compared with the other measures of impurity:
$$
  -\frac{1}{2} \left(\frac{9}{14} \log_{2} \frac{9}{14} + \frac{5}{14} \log_{2} \frac{5}{14}\right) = 0.47.
$$

### What is entropy?

To understand entropy, we need to take a detour to Shannon's theory of information.  The classic paper where these ideas are introduced is [Claude Shannon, *A mathematical theory of communication*, Bell System Technical Journal, 1948](http://ieeexplore.ieee.org/xpl/articleDetails.jsp?arnumber=6773024).

Shannon introduced the idea of the amount of information $I(p)$ gained by observing a random event with probability $p$.

Shannon's axioms of information:
1. Information is non-negative: $I(p) \geq 0$.
2. An event that always occurs provides no information: $I(1) = 0$.
3. Information from independent events is additive: $I(pq) = I(p) + I(q)$.

These axioms lead to 
$$
  I(p) = \log_{2} \frac{1}{p} = -\log_{2} p.
$$
Less likely events yield more information.

Given a discrete random variable $X$ with possible values $p_{1}, \ldots, p_{n}$, the **Shannon entropy** $H(X)$ is defined to be 
$$
  H(X) = - \sum_{i=1}^{n} p_{i} \log_{2} p_{i}.
$$
The entropy is a measure of the average number of bits needed to encode a randomly drawn value of $X$, under optimal encoding (e.g., Huffman encoding).

Optimal encoding uses longer codes for less likely values, using 
$\displaystyle -\log_{2} p_{i} = \log_{2} 1/p_{i}$ bits for $p_{i}$.

The expected number of bits to encode a value of $X$ is thus the entropy:
$$
  -\sum_{i=1}^{n} p_{i} \log_{2} p_{i}.
$$

**Shannon entropy: example.**  Consider the sequence AAAABBCD.  Let $p_{i}$ be the fraction of the time symbol $i$ appears.

For this sequence, the entropy is 
\begin{align*}
  -\left[ 
    \frac{1}{2} \log_{2} \frac{1}{2} 
    + \frac{1}{4} \log_{2} \frac{1}{4} 
    + \frac{1}{8} \log_{2} \frac{1}{8} 
    + \frac{1}{8} \log_{2} \frac{1}{8} 
  \right] &= \frac{1}{2} 
  + \frac{2}{4} 
  + \frac{3}{8}
  + \frac{3}{8} = \frac{7}{4}.
\end{align*}
This predicts that we can encode our sequence using $8 \times \frac{7}{4} = 14$ bits. 

A 16 bit encoding is 
$$
  \begin{array}{rclcrcl}
    A & = & 00, & \quad & C & = & 10 \\
    B & = & 01, &       & D & = & 11,
  \end{array}
$$
so 
$$
  \mbox{AAAABBCD} = (00)(00)(00)(00)(01)(01)(10)(11).
$$

Huffman encoding yields a 14 bit encoding by encoding the least common symbols with the longest codes:
$$
  \begin{array}{rclcrcl}
    A & = & 0, & \quad & C & = & 110 \\
    B & = & 10, &      & D & = & 111.
  \end{array}
$$
Then 
\begin{align*}
  \mbox{AAAABBCD} 
  &= \mbox{(0)(0)(0)(0)(10)(10)(110)(111)} \\
  &= \mbox{00001010110111}.
\end{align*}


### Entropy as a measure of (im)purity

**Back to decision trees&hellip;**  In the context of developing decision node tests, the entropy tells us the average number of bits needed to encode the classification of an arbitrary member of $S$ (i.e., a member of $S$ drawn at random).

If $S$ is homogeneous, we need fewer bits, so lower entropy indicates greater homogeneity.

## The two-class case

These measures of homogeneity are most easily understood (and visualized) in the two-class case.

Suppose we only have two classes, 1 and 2, in $S$.

If $p$ is the proportion of one of the classes in $S$, then $(1-p)$ is the proportion of the other class.

In this case we have
<table>
<tr><td>Misclassification rate:</td><td>$1 - \max(p, 1-p)$</td></tr>
<tr><td>Gini index:</td><td>$2 p (1 - p)$</td></tr>
<tr><td>Entropy:</td><td><div style="width:20em;">$-\frac{1}{2} \left(p \log_{2} p + (1-p) \log_{2} (1-p)\right)$</div></td></tr>
</table>

Notice that all three are zero when $S$ is homogeneous ($p = 0$ or $p = 1$), and at their maximum when $S$ is evenly split ($p = 1/2$).

In [ ]:
# Plot of the Gini index, the entropy, and the misclassification (error) rate.

import matplotlib.pyplot as plt
import numpy as np

def gini(p):
    return (p)*(1 - (p)) + (1-p)*(1 - (1-p))

def entropy(p):
    return -0.5 *(p*np.log2(p) + (1 - p)*np.log2((1 - p)))

def error(p):
    return 1 - np.max([p, 1 - p])

x = np.arange(0.0, 1.0, 0.01)

ent = [entropy(p) if p != 0 else None for p in x]
sc_ent = [e*0.5 if e else None for e in ent]
err = [error(i) for i in x]

fig = plt.figure()
ax = plt.subplot(111)
for i, lab, ls, c, in zip([ent, gini(x), err], 
                          ['Entropy', 'Gini index', 'Misclassification rate'],
                          ['-', '-', '--', '-.'],
                          ['black', 'red', 'green', 'cyan']):
    line = ax.plot(x, i, label=lab, linestyle=ls, lw=2, color=c)

ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15),
          ncol=3, fancybox=True, shadow=False)

ax.axhline(y=0.5, linewidth=1, color='k', linestyle='--')
ax.axhline(y=0.5, linewidth=1, color='k', linestyle='--')
plt.ylim([0, 0.5])
plt.xlabel('p(i=1)')
plt.ylabel('Impurity Index')
plt.tight_layout()
#plt.savefig('./figures/impurity.png', dpi=300, bbox_inches='tight')
plt.show()

# Information gain 

**Information gain** is a quantitative measure of the value of using an feature in a test, with the goal of producing more homogeneous groups as a result of testing on that feature.

Henceforth, we assume that we use entropy as our measure of homogeneity.

Let $A$ be an feature, $\mbox{values}(A)$ its possible values, and $S_{v}$ be the subset of $S$ for which $A$ has value $v$.

The information gain is defined to be
$$
  \newcommand{\abs}[1]{|\, #1 \,|}
  \mbox{gain}(S,A) = \mbox{entropy}(S) - \!\!\!\!\sum_{v \in \mbox{values}(S)} \frac{\abs{S_{v}}}{\abs{S}} \mbox{entropy}(S_{v}),
$$
where $\abs{\cdot}$ is the number of elements in a set.

The information gain is the expected reduction in entropy caused by knowing the value of the feature $A$.

**When constructing a decision node with training cases $S$, we want to choose the feature $A$ that maximizes the information gain.**

In the example we have been using we have two classes: cat and dog.  There are 9 cats and 5 dogs.

The initial entropy is $E = 0.470$:

In [ ]:
E = entropy(9/14)  # 9 cats out of 14 animals
print(f"{E:.4}")

Suppose we can now branch on one of the features <code>Height</code> and <code>Weight</code>.

Suppose <code>Height</code> leads to two nodes containing:
  1. 7 cats, 1 dog
  2. 2 cats, 4 dogs.

Let's look at the information gain:

In [ ]:
# Entropies of the two nodes resulting from a split based on height:
p = 7/8  # 7 cats out of 8 cases
entropy1 = entropy(p)
print(f"Entropy1 = {entropy1:.3}", )

p = 3/6  # 3 cats out of 6 cases
entropy2 = entropy(p)
print(f"Entropy2 = {entropy2:.3}", )

# Information gain:
gain = E - (8/14) * entropy1 - (6/14) * entropy2
print("Information gain = ", gain)

Meanwhile, suppose splitting based on <code>Weight</code> leads to two nodes containing:
  1. 3 cats, 3 dogs
  2. 6 cats, 2 dogs.

In [ ]:
# Entropies of the two nodes resulting from a split based on weight:
p = 3/6  # 3 cats out of 6 cases
entropy1 = entropy(p)
print(f"Entropy1 = {entropy1:.3}", )

p = 6/8  # 6 cats out of 8 cases
entropy2 = entropy(p)
print(f"Entropy2 = {entropy2:.3}", )

# Information gain:
gain = E - (6/14) * entropy1 - (8/14) * entropy2
print("Information gain = ", gain)

We see that the split on <code>Height</code> yields the greater information gain.

# Fisher's iris dataset

[Fisher's iris data set](https://en.wikipedia.org/wiki/Iris_flower_data_set) is a classic data set in statistics.  It first appears in [Sir Ronald Fisher's](http://www-history.mcs.st-andrews.ac.uk/Biographies/Fisher.html) 1936 paper in which he introduces the idea of **[linear discriminant analysis (LDA)](http://onlinelibrary.wiley.com/doi/10.1111/j.1469-1809.1936.tb02137.x/abstract;jsessionid=5A10931DC4623C414E3918FE030F64E0.f02t01)**.

The data set consists of 50 samples from each of three species of iris, 
* [*I. setosa*](https://en.wikipedia.org/wiki/Iris_setosa),
* [*I. versicolor*](https://en.wikipedia.org/wiki/Iris_versicolor)and 
* [*I. virginica*](https://en.wikipedia.org/wiki/Iris_virginica)
  
for a total of 150 instances.

Four **features** are measured for each sample: 
1. the length of the sepal,
2. the width of the sepal,
3. the length of the petal, and
4. the width of the petal.

The measurements are in centimeters.

The **class labels** are
1. Iris setosa,
2. Iris versicolor,
3. Iris virginica.

References:
* [R. A. Fisher, *The use of multiple measurements in taxonomic problems*, Annals of Eugenics, vol. 7, no. 2, pp. 179-188, 1936.](http://onlinelibrary.wiley.com/doi/10.1111/j.1469-1809.1936.tb02137.x/abstract;jsessionid=5A10931DC4623C414E3918FE030F64E0.f02t01)
In this paper Fisher develops a linear discriminant model that uses a combination of these four features to distinguish the species from one other.

Load the iris dataset from scikit-learn.

In [ ]:
from sklearn import datasets
import numpy as np

iris = datasets.load_iris()
X = iris.data
y = iris.target

First let's look at the class labels and names.   The classes are already converted to integer labels where
* 0 = Iris-Setosa,
* 1 = Iris-Versicolor, and
* 2 = Iris-Virginica.

In [ ]:
print("Class labels:", np.unique(y))
print("Class names:", iris.target_names)

Next we look at the feature data.  There is one instance per row:
* column 1: length of sepal
* column 2: width of sepal
* column 3: length of petal
* column 4: width of petal.

These are all in centimeters.

In [ ]:
print(iris.feature_names)
print(X[0:10,:])  # Just the first 10 rows.

# Building a decision tree in Scikit-Learn

The decision tree classifier in Scikit-Learn is called [DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

A helpful feature of decision trees is that they do not require much preprocessing of the data.  In particular, there is no need for us to scale the data.

We will use only the petal length and width so we will be able to plot the decision regions:

In [ ]:
X = X[:,2:]

Split the data into 70% training and 30% test data:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, 
    random_state=0)  # Be sure to set the random seed so the results are reproducible!

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
tree.fit(X_train, y_train)

Let's look at the decision regions.

In [ ]:
# A hacked up version of https://scikit-learn.org/stable/auto_examples/tree/plot_iris_dtc.html.

from sklearn.inspection import DecisionBoundaryDisplay

# Parameters
num_classes = 3
plot_colors = "ryb"

# Plot the decision boundaries.
plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)
DecisionBoundaryDisplay.from_estimator(
    tree,
    X,
    cmap=plt.cm.RdYlBu,
    response_method="predict",
    xlabel=iris.feature_names[2:],
    ylabel=iris.feature_names[2:],
)

# Plot the training points
for i, color in zip(range(num_classes), plot_colors):
    idx = np.where(y == i)
    plt.scatter(
        X[idx, 0],
        X[idx, 1],
        c=color,
        label=iris.target_names[i],
        edgecolor="black",
        s=15,
    )

plt.suptitle("Decision boundaries of the tree showing the training data")
plt.legend(loc="lower right", borderpad=0, handletextpad=0)
_ = plt.axis("tight")

## The accuracy of the tree

As a first pass at assessing the quality of our tree, let's look at the overall fraction of misclassified cases in both the training and test sets.

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = tree.predict(X_train)
print('Misclassified training samples: %d' % (y_train != y_pred).sum())
print('Accuracy: %.2f' % accuracy_score(y_train, y_pred))

In [ ]:
y_pred = tree.predict(X_test)
print('Misclassified test samples: %d' % (y_test != y_pred).sum())
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

We can do better by looking for patterns of misclassification using the [**confusion matrix**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html). 

In the confusion matrix $C = (C_{ij})$, the entry $C_{ij}$ is the number of cases in class $i$ that are predicted to be in class $j$.

In [ ]:
from sklearn.metrics import confusion_matrix

# The confusion matrix for the training data.
y_pred = tree.predict(X_train)

cm = confusion_matrix(y_train, y_pred)
print(cm)

We can present the confusion matrix in a fancier style:

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
class_names=("I. setosa", "I. versicolor", "I. virginica")
ConfusionMatrixDisplay.from_estimator(tree, X_test, y_test, display_labels=class_names)

## Examining the tree

The resulting tree looks like this.

<div style="text-align: center">
    <img src="http://www.cs.wm.edu/~rml/teaching/csci416/images/03_18.png"/>
</div>

This tree was produced using the <code>dot</code> utility from the [GraphViz](http://graphviz.org) package.

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(tree, 
                out_file='tree.dot', 
                feature_names=['petal length', 'petal width'])

!dot -Tpng tree.dot -o tree.png

Image(filename='tree.png')

# Continuous features <a id="continuous"/>

Continuous features can be converted to discrete features since we need a finite set of outcomes $O_{1}, \ldots, O_{n}$.

Temperature is continuous, but the value of the logical expression
$$
  \mbox{(temperature $<$ 32)}
$$
is discrete (true or false).

<table style="float:left;">
<tr><th>Forecast</th><th>Temp $(^{\circ}F)$</th><th>Humidity</th><th>Windy?</th><th>Class</th></tr>
<tr><td>sunny</td><td>75</td><td>70</td><td>true</td><td>Play</td></tr>
<tr><td>sunny</td><td>80</td><td>90</td><td>true</td><td>Don't Play</td></tr>
<tr><td>sunny</td><td>85</td><td>85</td><td>false</td><td>Don't Play</td></tr>
<tr><td>sunny</td><td>72</td><td>95</td><td>false</td><td>Don't Play</td></tr>
<tr><td>sunny</td><td>69</td><td>70</td><td>false</td><td>Play</td></tr>
<tr><td>cloudy</td><td>72</td><td>90</td><td>true</td><td>Play</td></tr>
<tr><td>cloudy</td><td>83</td><td>78</td><td>false</td><td>Play</td></tr>
<tr><td>cloudy</td><td>64</td><td>65</td><td>true</td><td>Play</td></tr>
<tr><td>cloudy</td><td>81</td><td>75</td><td>false</td><td>Play</td></tr>
<tr><td>rain</td><td>71</td><td>80</td><td>true</td><td>Don't Play</td></tr>
<tr><td>rain</td><td>65</td><td>70</td><td>true</td><td>Don't Play</td></tr>
<tr><td>rain</td><td>75</td><td>80</td><td>false</td><td>Play</td></tr>
<tr><td>rain</td><td>68</td><td>80</td><td>false</td><td>Play</td></tr>
<tr><td>rain</td><td>70</td><td>96</td><td>false</td><td>Play</td></tr>
</table>
<img src="http://www.cs.wm.edu/~rml/teaching/csci416/images/continuous_tree.png" style="width:50%;"/>

## Branching on continuous features

When devising tests on continuous features, we need to set a threshold to branch on 
(e.g., humidity $>$ 75%)

**We should choose the breakpoint to maximize information gain.**

If in our training data the continuous feature $A$ have the values 
$$
  a_{1} \leq a_{2} \leq \cdots a_{M},
$$
there are $M-1$ possible ways to split $A$ into $\{a_{1}, \ldots, a_{i}\}$ and $\{a_{i+1}, \ldots, a_{M}\}$.  All such splits are examined.

One choice of candidate thresholds is the midpoints
$$
  \frac{a_{i} + a_{i+1}}{2}.
$$
Alternatively, we could try the values $a_{i}$, which ensures that all threshold values come from the data.

## Geometric interpretation

The treatment of continuous features in decision trees has a geometric interpretation.
  
Suppose we have two continuous features $x_{1}, x_{2}$ that take on values between $-2$ and $2$, and two classes, Sheep and Goat.
<div style="text-align: center">
<img src="http://www.cs.wm.edu/~rml/teaching/csci416/images/sheep_vs_goat.jpg"/>
</div>

Suppose the decision tree is

<div style="text-align: center">
<img src="http://www.cs.wm.edu/~rml/teaching/csci416/images/sheep_goat_tree.png" style="height:400px"/>
</div>
The tree partitions the $(x_{1}, x_{2})$ domain: cases in the the green region are classified as Goats; cases in the yellow region are classified as Sheep.
<div style="text-align: center">
<img src="http://www.cs.wm.edu/~rml/teaching/csci416/images/tree_geometry1.png" style="height:400px"/>
</div>

If the true boundary between classes is not parallel to a coordinate axis, misclassification occurs:
<div style="text-align: center">
<img src="http://www.cs.wm.edu/~rml/teaching/csci416/images/tree_geometry2.png" style="height:400px"/>
</div>

# Missing values

It may be the case that for some of the cases in our training set have missing feature values.

One solution is to create a new feature value "missing" &ndash; we may find that cases with missing values for some feature behave differently than cases for this a value of the feature is present.

CART and C4.5/C5.0 also have general approaches to missing data.

In CART, whenever consider an feature for a split, only cases that have a value for that feature are considered.  CART then finds a hierarchy of **surrogate splits** using other features looking for a partition similar to the one found by excluding cases with the missing data.

If a case's outcome is not known for the original test, the outcome of the first surrogate is used; if that is also unknown, the next best surrogate is used, and so on. 

C4.5/C5.0 use a simpler approach.  They assume that unknown test outcomes are distributed probablistically according to the relative frequency of known outcomes.  

A case with an unknown test outcome is divided into fragments weighted according to these relative frequencies, so that a single case can follow multiple paths in the tree.

In this approach, information gain is modified as follows.  Given an feature $A$, we compute $\mbox{entropy}(S,A)$ and $\mbox{entropy}(S,A_{v})$ as before, except we only use the cases where the value of $A$ is present.

If $F$ is the fraction of cases for which the value of $A$ is known, then the modified information gain $\overline{\mbox{gain}}(S,A)$ is defined to be 
$$
  \overline{\mbox{gain}}(S,A) = F \times \mbox{gain}(S,A).
$$
This gives less weight to the information gain of an feature that is missing for a lot of cases.

In the initial training set $T$, all cases are given weight $w = 1$.

When a case from $T$ with known outcome $O_{i}$ and weight $w$ is assigned to $T_{i}$, it is given weight $w$.

Given a case with an unknown outcome, we assign it to $T_{i}$ with weight
$$
  w \times \mbox{probability of outcome $O_{i}$}.
$$
We estimate the probability by the relative frequency.

# Overfitting

The recursive partitioning of Hunt's method will continue to subdivide the set of training cases until either 
1. each subset in the partition contains cases of a single class, or
2. no test yields any improvement.

This may result in a very complicated tree that infers more structure (overfits the training data) than is justified by the training data.

**Overfitting** occurs when our model is describing random noise rather than true features, and leads to greater errors when making predictions about unseen cases.

## An example of overfitting

Construct a random training set.
* There are ten features, which take on the values 0 and 1 with equal probability.
* There are two classes; 75% have the value 0 (No) and 25% have the value 1 (Yes).
* Split the data into a training set of the first 500 cases and a test set of the last 500 cases.

In [ ]:
np.random.seed(42) # Make the result reproducible!

X = np.greater(np.random.rand(1000,10), 0.5).astype(float)
y = np.greater(np.random.rand(1000,1),  0.75).astype(float)
tree = DecisionTreeClassifier(criterion='entropy', random_state=0)
X_train = X[:500,:]
y_train = y[:500]
X_test = X[500:,:]
y_test = y[500:]
tree.fit(X_train, y_train)

y_pred = tree.predict(X_train)
print('Accuracy on training set: %.2f' % accuracy_score(y_train, y_pred))

y_pred = tree.predict(X_test)
print('Accuracy on test set:     %.2f' % accuracy_score(y_test, y_pred))

Scikit-learn builds a tree that correctly classifies 91% of the training set.

However, on the test set, only 66% are correctly classified.

On the other hand, a tree with a single node that classified everything as "No" would do better &ndash; it would be expected to correctly classify 75%!

Let's look at the resulting tree.  It is very complex.

In [ ]:
export_graphviz(tree, out_file='overfit_tree.dot')

!dot -Tpng overfit_tree.dot -o overfit_tree.png

Image(filename='overfit_tree.png') 

<img src="http://www.cs.wm.edu/~rml/teaching/csci416/images/overfit_tree.png" style="height:600px"/>

## What went wrong?

Suppose, in a two-class problem, $p$ is the proportion of cases belonging to the majority class ("No", in this case).

Consider two classifiers:
1. Classifier 1 assigns all cases to the majority class; its expected error rate is $1-p$.
2. Classifier 2 assigns a case to the majority class with probability $p$ and to the minority class with probability $1-p$; its expected error rate is the sum of
    * the probability that a case from the majority class is assigned to the minority class: $p(1-p)$, and
    * the probability that a case from the minority class is assigned to the majority class: $(1-p)p$.
    
The expected error rate for Classifier 2 is thus $2p(1-p)$.

This means that if $p > 0.5$, then $2p(1-p) > 1-p$, so Classifier 2 is expected to do worse than Classifier 1.

In our example with random data, the tree behaves a lot like classifier 2, sending each case randomly to leaf, which we would expect to be distributed according to the class frequencies in the training set.

For the random data, $p = 0.75$, so $2p(1-p) = 2 \times 0.75 \times 0.25 = 0.375$, which is close to the observed error rate of 34%.

The complex tree was overfit &ndash; it did a great job on the training data because it went to great efforts to create small leafs.

However, this structure was peculiar to the training data and did not generalize to the test data.

A simpler tree actually does better.  

In [ ]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_train)
print('Accuracy on training set: %.2f' % accuracy_score(y_train, y_pred))

y_pred = tree.predict(X_test)
print('Accuracy on test set:     %.2f' % accuracy_score(y_test, y_pred))

export_graphviz(tree, 
                out_file='not_overfit_tree.dot')

!dot -Tpng not_overfit_tree.dot -o not_overfit_tree.png

Image(filename='not_overfit_tree.png') 

Here is the resulting tree.

<img src="http://www.cs.wm.edu/~rml/teaching/csci416/images/not_overfit_tree.png"/>

# The bias-variance tradeoff

This example illustrates the **bias-variance tradeoff**.

**Bias** refers to the errors that occur because the model is too simple or just the wrong model for the data.

**Variance** refers to the errors that reflect sensitivity of the model to the training data.  Such sensivitity may be due to overfitting.

There is also likely to be an irreducible element of noise in the data that contributes to prediction errors.  Modeling the random errors leads to overfitting!

### High bias: diagnosis and treatment
The number 1 symptom of high bias is that the training error is unacceptably high.

Remedies include:
* Using a more complex model.
    * Introducing additional features to the model.
    * Changing the mathematical form of the model.
* Boosting (ensemble learning).

### High variance: diagnosis and treatment
The number 1 symptom of high variance is that the training error is much lower than the test error.

Remedies include
* Adding more training data.
* Reducing model complexity.
* Bagging (ensemble learning).

# Pruning <a id="pruning"/>

In order to have predictive power and reveal problem structure, we need a significant number of cases at each leaf.  This suggests we should prefer small (short) trees.

This approach is also consistent with a preference for the simplest model that fits the data (Occam's razor).

There are two ways to modify the recursive tree construction to produce simpler trees.
1. Prepruning: deciding not to partition a set of training cases any further.
2. Postpruning: Retrospectively removing some of the subtree structure built by recursive partitioning.

Stopping rules for prepruning may terminate partitioning before the benefits of subsequent partitioning becomes evident.  For this reason, CART and C4.5/C5.0 perform postpruning.

Decisions trees are usually pruned by discarding one or more subtrees and replacing them with leaves that contain the cases from the subtrees.  The class associated with a new leaf is the majority class of the training cases covered by the leaf.

This typically results in leaves that contain cases from more than one class, so more of the training cases will be misclassified. 
  
On the other hand, the resulting tree tend to be more accurate on unseen cases.

The presence of a distribution of classes in a leaf also leads to the perspective that we are computing probabilities that a training case belongs to that class.

### Reduced error postpruning
Suppose it were possible to predict the classification error rate of a tree and its subtrees.

Then we could prune as follows:
* Start from the bottom of the tree and examine each (nonleaf) subtree.
* If replacing a subtree by a leaf or the subtree's most frequently used branch would lead to
  predicted error rate that is no higher, then prune accordingly.
     * If the predicted error rate is not reduced, but not increased, either, we still would prune in order to simplify the tree.

Since the error rate of the entire tree decreases as the error rates of its subtrees are reduced, if we apply this process repeatedly then it will lead to a tree whose predicted error rate is minimal with respect to the allowable forms of pruning.

### Estimating classification error in postpruning
A simple way to estimate the classification error is to divide the set of training cases in two.

One set will be used to construct the decision tree.  We then apply the tree to the other set of data to estimate errors for the purposes of pruning.

The drawback is that some of the training data must be set aside and not used in constructing the tree.  If we have lots of training data, this is not a problem, but if data are scarce, it may result in an inferior tree.

Other approaches are based statistical resampling (cross-validation) and heuristic corrections to the error rate observed for the training data.

### Rule-based pruning
In addition to a decision tree, C4.5/C5.0 also produce an equivalent collection of rules.
The leftmost path in the tennis graph may be written as the rule
<code>
if (Outlook == Sunny) && (Humidity == High) {
  Play_Tennis = No
}
</code>

<img src="http://www.cs.wm.edu/~rml/teaching/csci416/images/tennis_tree.png" style="height:300px;"/>

Rule generation leads to another approach to pruning.
1. Build the decision tree, allowing overfitting to occur.
2. Convert the tree into an equivalent set of rules, creating one rule for each path from the root to a leaf. 
3. Prune each rule by removing any preconditions that result in improved estimated error rate for the rule.
4. Sort the pruned rules by their estimate accuracy, and consider them in this sorted sequence when classifiying.

In the example
<code>
  if (Outlook == Sunny) && (Humidity == High) {
    Play_Tennis = No
  }
</code>
rule-based postpruning would consider removing (Outlook == Sunny) and (Humidity == High).

Converting to a rule for each leaf allows us to distinguish between the different contexts in which a decision node is used.  We are able to prune each path individually.  

By contrast, if we prune nodes, then we affect all the decision paths that are in the subtree rooted at the node. 

Converting to rules (arguably) makes the decision process easier to understand.

# Regression trees

Decision trees can also be used for regression.  See the notes for a description of the details of constructing a regression tree.

The following example is a hacked up version of
<p>
https://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html#sphx-glr-auto-examples-tree-plot-tree-regression-py
</p>

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Create a random dataset.
rng = np.random.RandomState(42)
X = np.sort(5 * rng.rand(80, 1), axis=0)
y = np.sin(X).ravel()
y[::5] += 3 * (0.5 - rng.rand(16))

# Fit the regression model.
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(X, y)
regr_2.fit(X, y)

# Predict.
X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)

# Plot the results.
plt.figure()
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(X_test, y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test, y_2, color="yellowgreen", label="max_depth=5", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Two regression trees")
plt.legend()
plt.show()

The less complex tree illustrates bias.

The more complex tree illustrates variance (overfitting).  It is fitting the outliers which are due in this example to random noise we added.